In [5]:
from pathlib import Path
from methods import create_asset_gdf, create_request_json, get_result
from create_data_logger import logger


In [2]:
assets_csv="/Users/sgdowd/Documents/1_EODH/data/global_power_plant_database_with_assetcloss.csv"
continents_gpkg = './World_Continents_6670117379411214938.gpkg'
asset_classes = ["PowerGeneratingAsset","ThermalPowerGeneratingAsset","RealEstateAsset","IndustrialActivity"]
nos_of_rows = [10,500]
experiment_no = 1
base_dir = Path("./data")
assets_gdf = create_asset_gdf(assets_csv, continents_gpkg)


/Users/sgdowd/Documents/1_EODH/create_data/methods.py:31: DtypeWarning: Columns (6,12,38) have mixed types. Specify dtype option on import or set low_memory=False.
  assets_df = pd.read_csv(assets_csv)


In [3]:

for asset_class in asset_classes:
    for no_of_rows in nos_of_rows:
        logger.info(f"Running experiment {experiment_no} for {asset_class} with {no_of_rows} rows")
        asset_dir = base_dir / asset_class
        asset_dir.mkdir(exist_ok=True)
        output_dir = asset_dir/f"e{experiment_no}_{asset_class}_{no_of_rows}"
        output_dir.mkdir(exist_ok=True)
        request_dict = create_request_json(assets_gdf=assets_gdf, asset_class=asset_class, no_of_rows=no_of_rows, output_file=output_dir/"input.json")
        job_id = get_result(request_dict, output_dir/f"output_{asset_class}_{no_of_rows}.json")
        with open(output_dir/"job_id.txt", "w") as f:
            f.write(job_id)

        